# Phase II: Data Curation, Exploratory Analysis and Plotting



## Imports

In [1]:
import requests
import pandas as pd

## Getting the Data

In [38]:
api_key = "03bcc17f7d105b13199e0325b659d4ab"
page = 1

df = pd.DataFrame()
for i in range(50):
    url = f"https://api.themoviedb.org/3/movie/top_rated?api_key={api_key}&language=en-US&page={page}"

    response = requests.get(url)
    data = response.json()

    new_data = pd.DataFrame(data['results'])

    df = pd.concat([df, new_data])
    page += 1

def get_languages():
    """
    This function gets the list of movie languages from the TMDB.
    
    Returns:
        response (dict): The list of languages as a dictionary.
    """
    url = "https://api.themoviedb.org/3/configuration/languages"
    params = {"api_key": api_key}
    headers = {"accept": "application/json"}

    response = requests.get(url, headers=headers, params=params)
    return response.json()

def get_movie_genres():
    """
    This funciton gets the list of movie genres from the TMDB.
    
    Returns:
        response (dict): The list of genres as a dictionary.
    """
    url = "https://api.themoviedb.org/3/genre/movie/list?language=en"
    params = {"api_key": api_key}
    headers = {"accept": "application/json"}
    response = requests.get(url, headers=headers, params=params)

    return response.json()

df.head()


,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,/v8xVDqt8uCul3c3mgx4VpGCwxJC.jpg,"[18, 80]",278,en,The Shawshank Redemption,Imprisoned in the 1940s for the double murder ...,34.9294,/9cqNxx0GxF0bflZmeSMuL5tnGzr.jpg,1994-09-23,The Shawshank Redemption,False,8.712,29056
1,False,/jdHsptJbtalEuVhCV5i7kSC3g0x.jpg,"[18, 80]",238,en,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",32.1983,/3bhkrj58Vtu7enYsRolD1fZdja1.jpg,1972-03-14,The Godfather,False,8.686,21955
2,False,/kGzFbGhp99zva6oZODW5atUtnqi.jpg,"[18, 80]",240,en,The Godfather Part II,In the continuing saga of the Corleone crime f...,19.1140,/hek3koDUyRQk7FIhPXsa6mT2Zc3.jpg,1974-12-20,The Godfather Part II,False,8.572,13264
3,False,/zb6fM1CX41D9rF9hdgclu0peUmy.jpg,"[18, 36, 10752]",424,en,Schindler's List,The true story of how businessman Oskar Schind...,17.1857,/sF1U4EUQS8YHUYjNl3pMGNIQyr0.jpg,1993-12-15,Schindler's List,False,8.567,16786
4,False,/tj6iPnz18hGfr0LKqWmG6Cp3niO.jpg,[18],389,en,12 Angry Men,The defense and the prosecution have rested an...,13.2914,/ow3wq89wM8qd5X7hWKxiRfsFf9C.jpg,1957-04-10,12 Angry Men,False,8.548,9469


## Cleaning the Data

In [39]:
def clean_data(df):
    """
    Cleans the data given into these metrics: id, title, release year/month/day, popularity, language, genres, rating average, rating count. 

    Args:
        df (DataFrame): the inputted dataframe to be cleaned.

    Returns:
        cleaned_df (DataFrame): a cleaned dataframe.
    """
    
    #get the list of languages from TMDB
    languages_dict = get_languages()
    languages = {}
    for lang in languages_dict:
        languages[lang['iso_639_1']] = lang['english_name']
    
    #get the list of genres from TMDB
    genres_dict = get_movie_genres()
    genre_list = {}
    for genre in genres_dict['genres']:
        genre_list[genre['id']] = genre['name']

    #initialize lists to hold data
    movie_ids = []
    movie_titles = []
    movie_year = []
    movie_month = []
    movie_day = []
    movie_language = []
    movie_popularity = []
    movie_genres = []
    movie_vote_average = []
    movie_vote_count = []

    #loop through all rows of the data
    for data in df.iterrows():
        movie = data[1]

        movie_ids.append(movie['id']) #append movie id

        movie_titles.append(movie['title']) #append movie title

        #append movie release date
        movie_year.append(int(movie['release_date'][0:4])) 
        movie_month.append(int(movie['release_date'][5:7]))
        movie_day.append(int(movie['release_date'][8:10]))

        movie_language.append(languages[movie['original_language']]) #append movie language

        movie_popularity.append(float(movie['popularity'])) #append movie popularity

        #append movie genres as a list of genres
        genres = movie['genre_ids']
        list_of_genres = []
        for index in genres:
            list_of_genres.append(genre_list[index])
        movie_genres.append(list_of_genres)

        #append movie rating average and count
        movie_vote_average.append(float(movie['vote_average']))
        movie_vote_count.append(int(movie['vote_count']))

    #put all of the lists into a dictionary
    df_dict = {'movie_id': movie_ids,
               'title': movie_titles,
               'popularity': movie_popularity,
               'rating': movie_vote_average,
               'number_of_raters': movie_vote_count,
               'genres': movie_genres,
               'release_year': movie_year,
               'release_month': movie_month,
               'release_day': movie_day,
               'original_language': movie_language,
               }

    cleaned_df = pd.DataFrame(df_dict) #create the dataframe

    #make dummy variables for categorical data
    cleaned_df = pd.get_dummies(cleaned_df, columns=['original_language'], dtype='float')

    df_exploded = cleaned_df.explode('genres') #explode the genres column so each genre has its own row (we'll remove these extra rows later)
    df_genre_dummies = pd.get_dummies(df_exploded['genres']) #get the dummy variables for the genres
    cleaned_df = cleaned_df.join(df_genre_dummies.groupby(df_exploded.index).sum()) #put rows back together again

    cleaned_df = cleaned_df.drop(columns=['genres']) #drop the genres column now, as we don't need it

    cleaned_df = cleaned_df.sort_values(by='movie_id') #sort the dataframe by movie_id (this isn't really necessary)
    return cleaned_df.reset_index(drop=True) #return the dataframe, and reset the index of the rows

cleaned_df = clean_data(df)
cleaned_df

,movie_id,title,popularity,rating,number_of_raters,release_year,release_month,release_day,original_language_Arabic,original_language_Bengali,...,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
0,11,Star Wars,18.3734,8.204,21575,1977,5,25,0.0,0.0,...,0,0,0,0,0,1,0,0,0,0
1,12,Finding Nemo,15.5111,7.817,19990,2003,5,30,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,13,Forrest Gump,22.7920,8.464,28745,1994,6,23,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
3,14,American Beauty,8.5857,8.003,12601,1999,9,15,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,15,Citizen Kane,5.6597,8.001,5782,1941,4,17,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1244492,Look Back,11.8096,8.064,416,2024,6,28,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
996,1311031,Demon Slayer: Kimetsu no Yaiba Infinity Castle,261.8345,7.793,474,2025,7,18,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
997,1356039,Counterattack,9.2077,8.286,754,2025,2,27,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
998,1376434,Predator: Killer of Killers,24.7906,7.888,895,2025,6,5,0.0,0.0,...,0,0,0,0,0,1,0,1,0,0


## Visualizations